In [7]:
import faiss                   # make faiss available
import numpy as np
from os import listdir
from os.path import isfile, join
import pickle
import json
import hashlib
import pandas as pd
d = 1536                           # dimension



In [8]:
data_dir = '/Users/vli/Work/RetailSearch/RetailSearch/data/embeddings'
data_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.bin' in f]

data = []
for file in data_files:
    f = open(join(data_dir, file),'rb')
    data.append(pickle.load(f))
    # close the file
    f.close()

In [9]:
len(data[0])

2189

In [10]:
# d1 = data[0]

In [11]:
# np.where[d1[2]==None]

In [12]:
# data[0][2188]

In [14]:
embeddings = []
ids = []
for dt in data:
    for record in dt:
        embedding_size = len(record[2])
        assert embedding_size ==d , "embeeding size do not match {d} for {i}th row "
        embeddings.append(record[2])
        ids.append(record[1])

In [15]:
embeddings = np.array(embeddings)

In [16]:
ids = np.array(ids)

In [18]:
embeddings.shape

(17505, 1536)

In [20]:
n_bits = 2 * d
lsh = faiss.IndexLSH (d, n_bits)
lsh.train (embeddings)
lsh.add (embeddings)

In [24]:
index_path = 'index.faiss'

In [25]:
faiss.write_index(lsh, index_path)

In [26]:
new_index = faiss.read_index(index_path)

In [29]:
embeddings[10]

array([-0.0006499 , -0.02466407,  0.00072385, ...,  0.00288863,
       -0.01601001, -0.03824012])

In [34]:
D, I = new_index.search(embeddings[:1], 100)

array([[    0,    16,    12,   336,   354,   290,  2430,   370,  2363,
          355,   347,   321,  2333,  2368,  2407,   327,  2392,   301,
         2371,  2375,  2390,   329,  2395,  2452,   283,   344,    38,
         6060,  2440,   294,   333,   443,  6144,   444,  2382,  2383,
         3869,  2385,   293,  6103,   284,  2421,    52,   391,    51,
          309,   209,  2362,   337,  2685,   305,   381,   203,   340,
          345,  6061,   269,   747,  6084,  6124,    32,    43,  2400,
         2911,  5728,  6085,  6172,   408,   440,  2342,  2387,  2434,
         6086,   669,   693,  2771,   386,  2335,   325,  2345,  2408,
         2354, 13190,   705,   727,  4399,   349,  6064,  6129,  2862,
         2334,  2415,  2431,   204,   238,   298,   348,   297,  2743,
          458]])

In [38]:
data_dir = '/Users/vli/Work/RetailSearch/RetailSearch/data/'
data_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.json' in f]

In [39]:
def encode(text):
    hash_object = hashlib.md5(text.encode('utf-8'))
    return hash_object.hexdigest()

In [40]:
dfs = []
for data_file in data_files:
    input_file_path = join(data_dir,data_file)
    print(input_file_path)
    input_file = open(input_file_path)
    products = json.load(input_file)
    df_products = pd.DataFrame(products)
    dfs.append(df_products)
    input_file.close()

/Users/vli/Work/RetailSearch/RetailSearch/data/products_6.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_7.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_1.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_2.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_3.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_8.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_4.json
/Users/vli/Work/RetailSearch/RetailSearch/data/products_5.json


In [41]:
dfs = pd.concat(dfs)


In [44]:
dfs.iloc[0]

title                            Nu Finish Liquid Car Polish | 473mL
price                                                         $23.50
package_price                                        $4.97 per 100mL
product_image      https://productimages.coles.com.au/productimag...
product_details    <div class="sc-a2942c6c-0 kwRUuP product-detai...
url                https://www.coles.com.au/product/nu-finish-liq...
Name: 0, dtype: object

In [46]:
dfs.iloc[I[0]]

,title,price,package_price,product_image,product_details,url
0,Nu Finish Liquid Car Polish | 473mL,$23.50,$4.97 per 100mL,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/nu-finish-liq...
16,Coles Super Alkaline Battery 9V | 2 pack,$5.25,$2.63 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/coles-super-a...
12,NRG Super Alkaline AAA Batteries | 4 pack,$6.35,$1.59 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/nrg-super-alk...
336,Hot Topic Licensed Mug Adult 1 | 1 each,$6.50,$6.50 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/hot-topic-lic...
354,Smash Leak Proof Fruity Mover | 1 each,$6.50,$6.50 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/smash-leak-pr...
...,...,...,...,...,...,...
298,Korbond Lint Roller | 1 Each,$7.50,$7.50 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/korbond-lint-...
348,Korbond Polyester Thread Multi Colours | 5 pack,$4.20,$0.84 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/korbond-polye...
297,Pyrex Simply Store Glass Rectangle Container 7...,$6.90,$6.90 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/pyrex-simply-...
552,Coles Tampons Super | 32 pack,$3.75,$0.12 per 1ea,https://productimages.coles.com.au/productimag...,"<div class=""sc-a2942c6c-0 kwRUuP product-detai...",https://www.coles.com.au/product/coles-tampons...
